In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.prune as prune
from copy import deepcopy
import time
from ResNet import Bottleneck, ResNet, ResNet50

In [2]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:13<00:00, 12382601.44it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
net = ResNet50(10).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [19]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

Loss [1, 100](epoch, minibatch):  0.0526393671054393
Loss [1, 200](epoch, minibatch):  0.05295187274459749
Loss [1, 300](epoch, minibatch):  0.05301217177882791
Loss [2, 100](epoch, minibatch):  0.05588216789998114
Loss [2, 200](epoch, minibatch):  0.05273983641527593
Loss [2, 300](epoch, minibatch):  0.06406863600946963
Loss [3, 100](epoch, minibatch):  0.04777993245050311
Loss [3, 200](epoch, minibatch):  0.05010294391773641
Loss [3, 300](epoch, minibatch):  0.048276498820632695
Loss [4, 100](epoch, minibatch):  0.05068978223949671
Loss [4, 200](epoch, minibatch):  0.0482586153713055
Loss [4, 300](epoch, minibatch):  0.05181093121878803
Loss [5, 100](epoch, minibatch):  0.044640032006427646
Loss [5, 200](epoch, minibatch):  0.04661684276536107
Loss [5, 300](epoch, minibatch):  0.047416857935022566
Loss [6, 100](epoch, minibatch):  0.04445262691937387
Loss [6, 200](epoch, minibatch):  0.04324642017483711
Loss [6, 300](epoch, minibatch):  0.047380651608109475
Loss [7, 100](epoch, minib

### Before Pruning

In [63]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.49633002281188965
Accuracy on 10,000 test images:  86.85000000000001 %


In [64]:
pruned_model = deepcopy(net)

# 모든 레이어에 Unstructured Pruning 적용
def apply_pruning_to_all_layers(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=0.2)
            prune.remove(module, 'weight')
apply_pruning_to_all_layers(pruned_model)

In [69]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = pruned_model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.512721061706543
Accuracy on 10,000 test images:  86.0 %


In [70]:
pruned_model_2 = deepcopy(net)

# 모든 레이어에 Unstructured Pruning (amount=0.3) 적용
def apply_pruning_to_all_layers(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=0.3)
            prune.remove(module, 'weight')

apply_pruning_to_all_layers(pruned_model_2)

In [71]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = pruned_model_2(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.46253538131713867
Accuracy on 10,000 test images:  85.45 %


In [86]:
pruned_model_3 = deepcopy(net)

# 모든 레이어에 Unstructured Pruning (amount=0.1) 적용
def apply_pruning_to_all_layers(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=0.1)
            prune.remove(module, 'weight')

apply_pruning_to_all_layers(pruned_model_3)

In [87]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = pruned_model_3(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.4572627544403076
Accuracy on 10,000 test images:  86.77 %
